In [241]:
# -*- coding: utf-8 -*-
import pandas as pd
from pandas import DataFrame
import numpy as np
import json
import re
from __future__ import division
import string
import math
import nltk
from textblob import TextBlob as tb
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import twitter
import time


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shruthisubbaiah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Authentication for Twitter API

In [310]:
from twitter import *
auth = twitter.oauth.OAuth('962724602685394944-eiWoSq1e4P8ATMIm16KMw4SPA2jUrKF', 'FUMWZc2P2B5fJ9M0HXUcMMMifGyODyXdUFipgjTjruY2J',
                           '9qQwgXZjo0rYbfa1suN0bOiqD', '64GmL60QvLGm2IYDnCw4WMwnFaGzbm9ag0H6yXdETo7THPczZb')

twitter_api = twitter.Twitter(auth=auth)

# Reading the IMDB movie dataset into a datframe

In [311]:
movies_df=pd.read_csv("imdb_movie_dataset.csv")

#Choosing the required columns
movies_df=movies_df[['certificate','genre','gross_value','imdb_ratings','movie_names','star_cast','year_release']]
movies_df


,certificate,genre,gross_value,imdb_ratings,movie_names,star_cast,year_release
0,R,Action,226277068.0,8.1,Logan,James Mangold,2017
1,R,Drama,226277068.0,8.1,Logan,James Mangold,2017
2,R,fi,226277068.0,8.1,Logan,James Mangold,2017
3,R,Action,226277068.0,8.1,Logan,Hugh Jackman,2017
4,R,Drama,226277068.0,8.1,Logan,Hugh Jackman,2017
5,R,fi,226277068.0,8.1,Logan,Hugh Jackman,2017
6,R,Action,226277068.0,8.1,Logan,Patrick Stewart,2017
7,R,Drama,226277068.0,8.1,Logan,Patrick Stewart,2017
8,R,fi,226277068.0,8.1,Logan,Patrick Stewart,2017
9,R,Action,226277068.0,8.1,Logan,Dafne Keen,2017


# Cleaning the Movies dataframe

In [312]:
movies_df[movies_df.isnull().any(1)]


,certificate,genre,gross_value,imdb_ratings,movie_names,star_cast,year_release
311,TV-MA,Action,NaN,6.5,Bright,David Ayer,2017
312,TV-MA,Crime,NaN,6.5,Bright,David Ayer,2017
313,TV-MA,Fantasy,NaN,6.5,Bright,David Ayer,2017
314,TV-MA,Action,NaN,6.5,Bright,Will Smith,2017
315,TV-MA,Crime,NaN,6.5,Bright,Will Smith,2017
316,TV-MA,Fantasy,NaN,6.5,Bright,Will Smith,2017
317,TV-MA,Action,NaN,6.5,Bright,Joel Edgerton,2017
318,TV-MA,Crime,NaN,6.5,Bright,Joel Edgerton,2017
319,TV-MA,Fantasy,NaN,6.5,Bright,Joel Edgerton,2017
320,TV-MA,Action,NaN,6.5,Bright,Noomi Rapace,2017


In [313]:
movies_df.loc[movies_df['certificate'].isnull(),'certificate']='unknown'
movies_df[movies_df.isnull().any(1)]

,certificate,genre,gross_value,imdb_ratings,movie_names,star_cast,year_release
311,TV-MA,Action,NaN,6.5,Bright,David Ayer,2017
312,TV-MA,Crime,NaN,6.5,Bright,David Ayer,2017
313,TV-MA,Fantasy,NaN,6.5,Bright,David Ayer,2017
314,TV-MA,Action,NaN,6.5,Bright,Will Smith,2017
315,TV-MA,Crime,NaN,6.5,Bright,Will Smith,2017
316,TV-MA,Fantasy,NaN,6.5,Bright,Will Smith,2017
317,TV-MA,Action,NaN,6.5,Bright,Joel Edgerton,2017
318,TV-MA,Crime,NaN,6.5,Bright,Joel Edgerton,2017
319,TV-MA,Fantasy,NaN,6.5,Bright,Joel Edgerton,2017
320,TV-MA,Action,NaN,6.5,Bright,Noomi Rapace,2017


In [316]:
movies_df=movies_df[movies_df.gross_value.notnull()]
movies_df[movies_df.isnull().any(1)]

,certificate,genre,gross_value,imdb_ratings,movie_names,star_cast,year_release


In [317]:
movies_df

,certificate,genre,gross_value,imdb_ratings,movie_names,star_cast,year_release
0,R,Action,226277068.0,8.1,Logan,James Mangold,2017
1,R,Drama,226277068.0,8.1,Logan,James Mangold,2017
2,R,fi,226277068.0,8.1,Logan,James Mangold,2017
3,R,Action,226277068.0,8.1,Logan,Hugh Jackman,2017
4,R,Drama,226277068.0,8.1,Logan,Hugh Jackman,2017
5,R,fi,226277068.0,8.1,Logan,Hugh Jackman,2017
6,R,Action,226277068.0,8.1,Logan,Patrick Stewart,2017
7,R,Drama,226277068.0,8.1,Logan,Patrick Stewart,2017
8,R,fi,226277068.0,8.1,Logan,Patrick Stewart,2017
9,R,Action,226277068.0,8.1,Logan,Dafne Keen,2017


# Creating the Genres and Movie -Genres relationship tables

In [319]:
movie_genres=pd.DataFrame(columns=['movie_name','genre'])
i=1
for movie in movies:
    movie_temp=pd.DataFrame(movies_df[movies_df.movie_names==movie])
    for mt in movie_temp.itertuples():
        movie_genres.loc[str(i)]=[movie,mt[2]]
        i=i+1
    
movie_genres=movie_genres.drop_duplicates(keep='first')
movie_genres.loc[movie_genres['genre']=='fi','genre']="Sci-fi"
movie_genres

,movie_name,genre
1,Logan,Action
2,Logan,Drama
3,Logan,Sci-fi
13,Wonder Woman,Action
14,Wonder Woman,Adventure
15,Wonder Woman,Fantasy
25,Dunkirk,Action
26,Dunkirk,Drama
27,Dunkirk,History
37,Guardians of the Galaxy Vol. 2,Action


In [320]:
genres=movie_genres.genre.unique()
i=1
genre_df=pd.DataFrame(columns=['genre_name'])
for genre in genres:
    genre_df.loc[str(i)]=[genre]
    i=i+1
genre_df.loc[genre_df['genre_name']=='fi','genre_name']="Sci-fi"
genre_df

,genre_name
1,Action
2,Drama
3,Sci-fi
4,Adventure
5,Fantasy
6,History
7,Crime
8,Music
9,Horror
10,Mystery


# Creating the Actors and Movie-Actor relationship table

In [326]:
movie_actors=pd.DataFrame(columns=['movie_name','actor_name'])
i=1
for movie in movies:
    actor_temp=pd.DataFrame(movies_df[movies_df.movie_names==movie])
    for actor in actor_temp.itertuples():
        movie_actors.loc[str(i)]=[movie,actor[6]]
        i=i+1
    
movie_actors=movie_actors.drop_duplicates(keep='first')
movie_actors

,movie_name,actor_name
1,Logan,James Mangold
4,Logan,Hugh Jackman
7,Logan,Patrick Stewart
10,Logan,Dafne Keen
13,Wonder Woman,Patty Jenkins
16,Wonder Woman,Gal Gadot
19,Wonder Woman,Chris Pine
22,Wonder Woman,Robin Wright
25,Dunkirk,Christopher Nolan
28,Dunkirk,Fionn Whitehead


In [327]:
actors=movie_actors.actor_name.unique()
i=1
actor_df=pd.DataFrame(columns=['actor_name'])
for actor in actors:
    actor_df.loc[str(i)]=[actor]
    i=i+1

actor_df



,actor_name
1,James Mangold
2,Hugh Jackman
3,Patrick Stewart
4,Dafne Keen
5,Patty Jenkins
6,Gal Gadot
7,Chris Pine
8,Robin Wright
9,Christopher Nolan
10,Fionn Whitehead


In [328]:
movies_df=pd.DataFrame(movies_df,columns=['certificate','gross_value','imdb_ratings','movie_names','year_release'])
movies_df.drop_duplicates(keep='first')

,certificate,gross_value,imdb_ratings,movie_names,year_release
0,R,226277068.0,8.1,Logan,2017
12,PG-13,412563408.0,7.5,Wonder Woman,2017
24,PG-13,188373161.0,8.0,Dunkirk,2017
36,PG-13,389813101.0,7.7,Guardians of the Galaxy Vol. 2,2017
48,PG-13,618375718.0,7.5,Star Wars: Episode VIII - The Last Jedi,2017
60,PG-13,334201140.0,7.5,Spider-Man: Homecoming,2017
72,R,107825862.0,7.7,Baby Driver,2017
84,R,176040665.0,7.7,Get Out,2017
96,R,327481748.0,7.5,It,2017
108,R,92054159.0,8.2,Blade Runner 2049,2017


In [329]:
movies_df=movies_df.drop_duplicates(subset='movie_names',keep='first')

In [330]:
movies_df

,certificate,gross_value,imdb_ratings,movie_names,year_release
0,R,226277068.0,8.1,Logan,2017
12,PG-13,412563408.0,7.5,Wonder Woman,2017
24,PG-13,188373161.0,8.0,Dunkirk,2017
36,PG-13,389813101.0,7.7,Guardians of the Galaxy Vol. 2,2017
48,PG-13,618375718.0,7.5,Star Wars: Episode VIII - The Last Jedi,2017
60,PG-13,334201140.0,7.5,Spider-Man: Homecoming,2017
72,R,107825862.0,7.7,Baby Driver,2017
84,R,176040665.0,7.7,Get Out,2017
96,R,327481748.0,7.5,It,2017
108,R,92054159.0,8.2,Blade Runner 2049,2017


In [331]:
movies=movies_df.movie_names.unique()
len(movies)

195

# Using the Twitter API to run searches on each movie name

In [332]:
count =1

sep = ':'
tweets=pd.DataFrame(columns=['movie_name','user_id','tweet_text','hashtags','user_mentions','created_date','retweet_count','screen_name'])
i=1
for movie in movies:
    if count>=100:
        time.sleep(60*15)
        count=1
        auth = twitter.oauth.OAuth('962724602685394944-eiWoSq1e4P8ATMIm16KMw4SPA2jUrKF', 'FUMWZc2P2B5fJ9M0HXUcMMMifGyODyXdUFipgjTjruY2J',
                           '9qQwgXZjo0rYbfa1suN0bOiqD', '64GmL60QvLGm2IYDnCw4WMwnFaGzbm9ag0H6yXdETo7THPczZb')

        twitter_api = twitter.Twitter(auth=auth)
        
    movie=movie.replace(' ','')
    movie=movie.split(sep, 1)[0]
    q='#'+movie
    search_results = twitter_api.search.tweets(q=q,count=100)
    count=count+1
    statuses=json.dumps(search_results['statuses'])
    stat=json.loads(statuses)
    for s in stat:
        hashtags=''
        user_mentions=''
        for hashtag in s['entities']['hashtags']:
            hashtags=hashtags+' '+ hashtag['text']
        for user_mention in s['entities']['user_mentions']:
            user_mentions = user_mentions+' '+ user_mention['screen_name']
        tweets.loc[str(i)]=[movie,s['user']['id'],s['text'],hashtags,user_mentions,s['created_at'],s['retweet_count'],s['user']['screen_name']]
        i=i+1
tweets


,movie_name,user_id,tweet_text,hashtags,user_mentions,created_date,retweet_count,screen_name
1,Logan,738139568076001280,@Sasori_113 @LaraJillMiller @missnotyou @Matth...,,Sasori_113 LaraJillMiller missnotyou MatthewB...,Wed Feb 28 16:29:12 +0000 2018,0,AarontheWise
2,Logan,1398267104,Great catching up with Young Charlotte aka @as...,dareto bebold logan,ashlyncasalegno,Wed Feb 28 16:26:20 +0000 2018,0,agent_joechavez
3,Logan,143545821,"#Cine 🎬| #Oscars 2018: Sí a #Logan, pero no a...",Cine Oscars Logan WonderWoman,canaltn8,Wed Feb 28 16:23:05 +0000 2018,0,canaltn8
4,Logan,959474670864158720,RT @ChanguitoGol: Logan 😭😭😭😭😭\n#Wolverine...,Wolverine logan Marvel,ChanguitoGol,Wed Feb 28 16:15:19 +0000 2018,1,Favs_Pops
5,Logan,211299432,Who told you that CBM can't be thematically re...,SeriousCBM Logan XMen,,Wed Feb 28 16:11:36 +0000 2018,0,scedydesigner
6,Logan,119591408,"See our latest #Logan, UT #job and click to ap...",Logan job Labor Hiring CareerArc,,Wed Feb 28 16:07:09 +0000 2018,0,tmj_UT_LABOR
7,Logan,833612154,Films like #Logan and #BlackPanther could lead...,Logan BlackPanther Oscars,,Wed Feb 28 16:06:03 +0000 2018,0,APEntertainment
8,Logan,117789928,We're #hiring! Read about our latest #job open...,hiring job,,Wed Feb 28 16:05:11 +0000 2018,0,tmj_ut_intern
9,Logan,3018132191,Das Ende der X-Men Filme.. Sehr gut 👍 kann ic...,logan xmen Logan4Life xmen_wolverine end,,Wed Feb 28 16:04:32 +0000 2018,0,steffimaus81
10,Logan,149112383,We're #hiring! Read about our latest #job open...,hiring job KellyJobs,,Wed Feb 28 16:00:46 +0000 2018,0,tmj_UT_transp


# Cleaning the Tweets dataframe

In [333]:
tweets


,movie_name,user_id,tweet_text,hashtags,user_mentions,created_date,retweet_count,screen_name
1,Logan,738139568076001280,@Sasori_113 @LaraJillMiller @missnotyou @Matth...,,Sasori_113 LaraJillMiller missnotyou MatthewB...,Wed Feb 28 16:29:12 +0000 2018,0,AarontheWise
2,Logan,1398267104,Great catching up with Young Charlotte aka @as...,dareto bebold logan,ashlyncasalegno,Wed Feb 28 16:26:20 +0000 2018,0,agent_joechavez
3,Logan,143545821,"#Cine 🎬| #Oscars 2018: Sí a #Logan, pero no a...",Cine Oscars Logan WonderWoman,canaltn8,Wed Feb 28 16:23:05 +0000 2018,0,canaltn8
4,Logan,959474670864158720,RT @ChanguitoGol: Logan 😭😭😭😭😭\n#Wolverine...,Wolverine logan Marvel,ChanguitoGol,Wed Feb 28 16:15:19 +0000 2018,1,Favs_Pops
5,Logan,211299432,Who told you that CBM can't be thematically re...,SeriousCBM Logan XMen,,Wed Feb 28 16:11:36 +0000 2018,0,scedydesigner
6,Logan,119591408,"See our latest #Logan, UT #job and click to ap...",Logan job Labor Hiring CareerArc,,Wed Feb 28 16:07:09 +0000 2018,0,tmj_UT_LABOR
7,Logan,833612154,Films like #Logan and #BlackPanther could lead...,Logan BlackPanther Oscars,,Wed Feb 28 16:06:03 +0000 2018,0,APEntertainment
8,Logan,117789928,We're #hiring! Read about our latest #job open...,hiring job,,Wed Feb 28 16:05:11 +0000 2018,0,tmj_ut_intern
9,Logan,3018132191,Das Ende der X-Men Filme.. Sehr gut 👍 kann ic...,logan xmen Logan4Life xmen_wolverine end,,Wed Feb 28 16:04:32 +0000 2018,0,steffimaus81
10,Logan,149112383,We're #hiring! Read about our latest #job open...,hiring job KellyJobs,,Wed Feb 28 16:00:46 +0000 2018,0,tmj_UT_transp


In [334]:
tweets['created_date'] = pd.to_datetime(tweets['created_date'], infer_datetime_format=True)
tweets

,movie_name,user_id,tweet_text,hashtags,user_mentions,created_date,retweet_count,screen_name
1,Logan,738139568076001280,@Sasori_113 @LaraJillMiller @missnotyou @Matth...,,Sasori_113 LaraJillMiller missnotyou MatthewB...,2018-02-28 16:29:12,0,AarontheWise
2,Logan,1398267104,Great catching up with Young Charlotte aka @as...,dareto bebold logan,ashlyncasalegno,2018-02-28 16:26:20,0,agent_joechavez
3,Logan,143545821,"#Cine 🎬| #Oscars 2018: Sí a #Logan, pero no a...",Cine Oscars Logan WonderWoman,canaltn8,2018-02-28 16:23:05,0,canaltn8
4,Logan,959474670864158720,RT @ChanguitoGol: Logan 😭😭😭😭😭\n#Wolverine...,Wolverine logan Marvel,ChanguitoGol,2018-02-28 16:15:19,1,Favs_Pops
5,Logan,211299432,Who told you that CBM can't be thematically re...,SeriousCBM Logan XMen,,2018-02-28 16:11:36,0,scedydesigner
6,Logan,119591408,"See our latest #Logan, UT #job and click to ap...",Logan job Labor Hiring CareerArc,,2018-02-28 16:07:09,0,tmj_UT_LABOR
7,Logan,833612154,Films like #Logan and #BlackPanther could lead...,Logan BlackPanther Oscars,,2018-02-28 16:06:03,0,APEntertainment
8,Logan,117789928,We're #hiring! Read about our latest #job open...,hiring job,,2018-02-28 16:05:11,0,tmj_ut_intern
9,Logan,3018132191,Das Ende der X-Men Filme.. Sehr gut 👍 kann ic...,logan xmen Logan4Life xmen_wolverine end,,2018-02-28 16:04:32,0,steffimaus81
10,Logan,149112383,We're #hiring! Read about our latest #job open...,hiring job KellyJobs,,2018-02-28 16:00:46,0,tmj_UT_transp


In [335]:
tweets['tweet_id']=tweets.index
tweets


,movie_name,user_id,tweet_text,hashtags,user_mentions,created_date,retweet_count,screen_name,tweet_id
1,Logan,738139568076001280,@Sasori_113 @LaraJillMiller @missnotyou @Matth...,,Sasori_113 LaraJillMiller missnotyou MatthewB...,2018-02-28 16:29:12,0,AarontheWise,1
2,Logan,1398267104,Great catching up with Young Charlotte aka @as...,dareto bebold logan,ashlyncasalegno,2018-02-28 16:26:20,0,agent_joechavez,2
3,Logan,143545821,"#Cine 🎬| #Oscars 2018: Sí a #Logan, pero no a...",Cine Oscars Logan WonderWoman,canaltn8,2018-02-28 16:23:05,0,canaltn8,3
4,Logan,959474670864158720,RT @ChanguitoGol: Logan 😭😭😭😭😭\n#Wolverine...,Wolverine logan Marvel,ChanguitoGol,2018-02-28 16:15:19,1,Favs_Pops,4
5,Logan,211299432,Who told you that CBM can't be thematically re...,SeriousCBM Logan XMen,,2018-02-28 16:11:36,0,scedydesigner,5
6,Logan,119591408,"See our latest #Logan, UT #job and click to ap...",Logan job Labor Hiring CareerArc,,2018-02-28 16:07:09,0,tmj_UT_LABOR,6
7,Logan,833612154,Films like #Logan and #BlackPanther could lead...,Logan BlackPanther Oscars,,2018-02-28 16:06:03,0,APEntertainment,7
8,Logan,117789928,We're #hiring! Read about our latest #job open...,hiring job,,2018-02-28 16:05:11,0,tmj_ut_intern,8
9,Logan,3018132191,Das Ende der X-Men Filme.. Sehr gut 👍 kann ic...,logan xmen Logan4Life xmen_wolverine end,,2018-02-28 16:04:32,0,steffimaus81,9
10,Logan,149112383,We're #hiring! Read about our latest #job open...,hiring job KellyJobs,,2018-02-28 16:00:46,0,tmj_UT_transp,10


In [336]:
tweets.loc[tweets['screen_name'].isnull(),'screen_name']='unknown'
tweets[tweets.isnull().any(1)]

,movie_name,user_id,tweet_text,hashtags,user_mentions,created_date,retweet_count,screen_name,tweet_id


In [337]:
#Writing into a csv file

tweets.to_csv("Tweets.csv",encoding='utf-8')



In [338]:
tweets_df=pd.DataFrame(tweets,columns=['tweet_id','tweet_text','hashtags','user_mentions','created_date','retweet_count'])


# Creating the Users dataframe

In [339]:
users_df=pd.DataFrame(tweets,columns=['user_id','screen_name'])
users_df=users_df.drop_duplicates(subset='user_id',keep='first')
users_df

,user_id,screen_name
1,738139568076001280,AarontheWise
2,1398267104,agent_joechavez
3,143545821,canaltn8
4,959474670864158720,Favs_Pops
5,211299432,scedydesigner
6,119591408,tmj_UT_LABOR
7,833612154,APEntertainment
8,117789928,tmj_ut_intern
9,3018132191,steffimaus81
10,149112383,tmj_UT_transp


# Creating the dataframe for Users and their Tweets

In [340]:
user_posts=pd.DataFrame(tweets,columns=['user_id','tweet_id'])
user_posts


,user_id,tweet_id
1,738139568076001280,1
2,1398267104,2
3,143545821,3
4,959474670864158720,4
5,211299432,5
6,119591408,6
7,833612154,7
8,117789928,8
9,3018132191,9
10,149112383,10


# Retrieve tags from each tweet

In [341]:

from nltk import wordpunct_tokenize
from string import punctuation
from nltk.probability import FreqDist
from nltk import pos_tag
from nltk import word_tokenize

punc=punctuation+'\n'+'\t'+'rt'+'RT'
stop = stopwords.words('english')
from collections import Counter
i=1
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punc)

def eliminate_stopwords(review):
    str=""
    word = word_tokenize(review)
    tagged=nltk.pos_tag(word)
    for t in tagged:
        if t[1] !='ADJ' and t[1]!='RB' and t[1]!='DT'and t[1]!='IN' and t[1]!='TO' and t[1]!='PRP'and t[1]!='CC' and t[0] not in stop:
            str = str + ' '+ t[0]
    return str


In [343]:
#movie_tags_df=pd.DataFrame(columns=['movie_name','tags'])
tweet_tags_df=pd.DataFrame(columns=['Tweet_id','tags'])
i=1
for tweet in tweets.itertuples():
    tag_list=[]
    for item in [tweet[4].strip().split(' '), tweet[5].strip().split(' ')]:
        #c = Counter(item)
        #tag_list.append(c.most_common(5))
        tag_list.append(item)

    for tl in tag_list:
        for t in tl:
            tweet_tags_df.loc[str(i)]=[tweet[9],t.lower()]
            #movie_tags_df.loc[str(i)]=[tweet[1],t[0].lower()]
            i=i+1
#movie_tags_df.drop_duplicates(keep='first')
tweet_tags_df



,Tweet_id,tags
1,1,
2,1,sasori_113
3,1,larajillmiller
4,1,missnotyou
5,1,matthewbouman1
6,1,metalskull423
7,1,brutalpuncher1
8,1,jjravenation52
9,1,davidcart11
10,2,dareto


In [345]:
tweet_tags_df=tweet_tags_df.drop_duplicates(keep='first')
tweet_tags_df[tweet_tags_df.isnull().any(1)]

,Tweet_id,tags


# Creating the Tweets table

In [346]:
tweets_table=pd.DataFrame(columns=['tweet_id','tweet_text','hashtags','user_mentions','created_date','retweet_count'])
from string import punctuation
punc=punctuation+'\n'+'\t'
i=1
for tweet in tweets_df.itertuples():
    text=tweet[2]
    text= strip_punctuation(text)
    text=text.lower().strip()
    tweets_table.loc[str(i)]=[tweet[1],text,tweet[3],tweet[4],tweet[5],tweet[6]]
    i=i+1
tweets_table


,tweet_id,tweet_text,hashtags,user_mentions,created_date,retweet_count
1,1,sasori113 larajillmiller missnotyou matthewbou...,,Sasori_113 LaraJillMiller missnotyou MatthewB...,2018-02-28 16:29:12,0
2,2,great catching up with young charlotte aka ash...,dareto bebold logan,ashlyncasalegno,2018-02-28 16:26:20,0
3,3,cine 🎬 oscars 2018 sí a logan pero no a wonde...,Cine Oscars Logan WonderWoman,canaltn8,2018-02-28 16:23:05,0
4,4,rt changuitogol logan 😭😭😭😭😭wolverine loga...,Wolverine logan Marvel,ChanguitoGol,2018-02-28 16:15:19,1
5,5,who told you that cbm cant be thematically rel...,SeriousCBM Logan XMen,,2018-02-28 16:11:36,0
6,6,see our latest logan ut job and click to apply...,Logan job Labor Hiring CareerArc,,2018-02-28 16:07:09,0
7,7,films like logan and blackpanther could lead t...,Logan BlackPanther Oscars,,2018-02-28 16:06:03,0
8,8,were hiring read about our latest job opening ...,hiring job,,2018-02-28 16:05:11,0
9,9,das ende der xmen filme sehr gut 👍 kann ich n...,logan xmen Logan4Life xmen_wolverine end,,2018-02-28 16:04:32,0
10,10,were hiring read about our latest job opening ...,hiring job KellyJobs,,2018-02-28 16:00:46,0


# Creating the movie-tweets relationship table

In [347]:
movie_tweet=pd.DataFrame(tweets,columns=['movie_name','tweet_id'])
movie_tweet


,movie_name,tweet_id
1,Logan,1
2,Logan,2
3,Logan,3
4,Logan,4
5,Logan,5
6,Logan,6
7,Logan,7
8,Logan,8
9,Logan,9
10,Logan,10


# Making a connection to the SQLite database

In [372]:
import sqlite3
conn = sqlite3.connect('Shruthi_Assignment2.db') 
c = conn.cursor()
conn.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

SQL Queries to create the tables:
    
    MOVIES table: Create table Movies(certificate varchar(10) not null, gross_value float not null, imdb_ratings float not null,movie_names varchar(30) PRIMARY KEY,year_release datetime not null)
    
    GENRES table: Create table Genres(genre_name varchar(25) PRIMARY KEY)
        
    MOVIE_GENRES table: Create table Movie_genres(movie_name varchar(30) not null,genre varchar(30) not null, PRIMARY KEY(movie_name,genre))
        
    ACTORS table: Create table Actors(actor_name varchar(20) PRIMARY KEY)
        
    MOVIE_ACTORS table: Create table Movie_actors(movie_name varchar(30) not null,actor_name varchar(30) not null,PRIMARY KEY(movie_name,actor_name))
        
    TWEETS table: Create table Tweets(tweet_id int PRIMARY KEY,tweet_text varchar(300) not null, hashtags varchar(30),user_mentions varchar(30),created_date datetime not null,retweet_count int not null )
        
    MOVIE_TWEET table: Create table Movie_tweet(movie_name varchar(30) not null, tweet_id int not null,PRIMARY KEY(movie_name,tweet_id))
        
    USERS table: Create table Users(user_id longint PRIMARY KEY,screen_name varchar(20))
        
    USER_POSTS table: Create table user_posts(user_id int not null,tweet_id int not null,PRIMARY KEY(user_id,tweet_id))
    
    TWEETS_TAGS table: Create table Tweets_tags(Tweet_id int not null,tags varchar(30),PRIMARY KEY(Tweet_id,tags))

# Create the tables in the database

In [373]:
c.execute("Create table Movies(certificate varchar(10) not null, gross_value float not null, imdb_ratings float not null,movie_names varchar(30) PRIMARY KEY,year_release datetime not null)")
movies_df.to_sql('Movies',con=conn,if_exists='append',index=False)

In [374]:
c.execute("Create table Genres(genre_name varchar(25) PRIMARY KEY)")
genre_df.to_sql('Genres',con=conn,if_exists='append',index=False)


In [375]:
c.execute("Create table Movie_genres(movie_name varchar(30) not null,genre varchar(30) not null, PRIMARY KEY(movie_name,genre))")
movie_genres.to_sql('Movie_genres',con=conn,if_exists='append',index=False)


In [376]:
c.execute("Create table Actors(actor_name varchar(20) PRIMARY KEY)")
actor_df.to_sql('Actors',con=conn,if_exists='append',index=False)

In [377]:
c.execute("Create table Movie_actors(movie_name varchar(30) not null,actor_name varchar(30) not null,PRIMARY KEY(movie_name,actor_name))")
movie_actors.to_sql('Movie_actors',con=conn,if_exists='append',index=False)

In [378]:
c.execute("Create table Tweets(tweet_id int PRIMARY KEY,tweet_text varchar(300) not null, hashtags varchar(30),user_mentions varchar(30),created_date datetime not null,retweet_count int not null )")
tweets_table.to_sql('Tweets',con=conn,if_exists='append',index=False)

In [379]:
c.execute("Create table Movie_tweet(movie_name varchar(30) not null, tweet_id int not null,PRIMARY KEY(movie_name,tweet_id))")
movie_tweet.to_sql('Movie_tweet',con=conn,if_exists='append',index=False)

In [380]:
c.execute("Create table Users(user_id longint PRIMARY KEY,screen_name varchar(20))")
users_df.to_sql('Users',con=conn,if_exists='append',index=False)


In [381]:
c.execute("Create table user_posts(user_id int not null,tweet_id int not null,PRIMARY KEY(user_id,tweet_id))")
user_posts.to_sql('user_posts',con=conn,if_exists='append',index=False)


In [382]:
c.execute("Create table Tweets_tags(Tweet_id int not null,tags varchar(30),PRIMARY KEY(Tweet_id,tags))")
tweet_tags_df.to_sql('Tweets_tags',con=conn,if_exists='append',index=False)


# View each table

In [383]:
pd.read_sql_query("SELECT * from Movies",con=conn)


,certificate,gross_value,imdb_ratings,movie_names,year_release
0,R,226277068.0,8.1,Logan,2017
1,PG-13,412563408.0,7.5,Wonder Woman,2017
2,PG-13,188373161.0,8.0,Dunkirk,2017
3,PG-13,389813101.0,7.7,Guardians of the Galaxy Vol. 2,2017
4,PG-13,618375718.0,7.5,Star Wars: Episode VIII - The Last Jedi,2017
5,PG-13,334201140.0,7.5,Spider-Man: Homecoming,2017
6,R,107825862.0,7.7,Baby Driver,2017
7,R,176040665.0,7.7,Get Out,2017
8,R,327481748.0,7.5,It,2017
9,R,92054159.0,8.2,Blade Runner 2049,2017


In [384]:
pd.read_sql_query("SELECT * from Genres",con=conn)

,genre_name
0,Action
1,Drama
2,Sci-fi
3,Adventure
4,Fantasy
5,History
6,Crime
7,Music
8,Horror
9,Mystery


In [385]:
pd.read_sql_query("SELECT * from Movie_genres",con=conn)

,movie_name,genre
0,Logan,Action
1,Logan,Drama
2,Logan,Sci-fi
3,Wonder Woman,Action
4,Wonder Woman,Adventure
5,Wonder Woman,Fantasy
6,Dunkirk,Action
7,Dunkirk,Drama
8,Dunkirk,History
9,Guardians of the Galaxy Vol. 2,Action


In [386]:
pd.read_sql_query("SELECT * from Actors",con=conn)

,actor_name
0,James Mangold
1,Hugh Jackman
2,Patrick Stewart
3,Dafne Keen
4,Patty Jenkins
5,Gal Gadot
6,Chris Pine
7,Robin Wright
8,Christopher Nolan
9,Fionn Whitehead


In [387]:
pd.read_sql_query("SELECT * from Movie_actors",con=conn)

,movie_name,actor_name
0,Logan,James Mangold
1,Logan,Hugh Jackman
2,Logan,Patrick Stewart
3,Logan,Dafne Keen
4,Wonder Woman,Patty Jenkins
5,Wonder Woman,Gal Gadot
6,Wonder Woman,Chris Pine
7,Wonder Woman,Robin Wright
8,Dunkirk,Christopher Nolan
9,Dunkirk,Fionn Whitehead


In [388]:
pd.read_sql_query("SELECT * from Tweets",con=conn)

,tweet_id,tweet_text,hashtags,user_mentions,created_date,retweet_count
0,1,sasori113 larajillmiller missnotyou matthewbou...,,Sasori_113 LaraJillMiller missnotyou MatthewB...,2018-02-28 16:29:12,0
1,2,great catching up with young charlotte aka ash...,dareto bebold logan,ashlyncasalegno,2018-02-28 16:26:20,0
2,3,cine 🎬 oscars 2018 sí a logan pero no a wonde...,Cine Oscars Logan WonderWoman,canaltn8,2018-02-28 16:23:05,0
3,4,rt changuitogol logan 😭😭😭😭😭wolverine loga...,Wolverine logan Marvel,ChanguitoGol,2018-02-28 16:15:19,1
4,5,who told you that cbm cant be thematically rel...,SeriousCBM Logan XMen,,2018-02-28 16:11:36,0
5,6,see our latest logan ut job and click to apply...,Logan job Labor Hiring CareerArc,,2018-02-28 16:07:09,0
6,7,films like logan and blackpanther could lead t...,Logan BlackPanther Oscars,,2018-02-28 16:06:03,0
7,8,were hiring read about our latest job opening ...,hiring job,,2018-02-28 16:05:11,0
8,9,das ende der xmen filme sehr gut 👍 kann ich n...,logan xmen Logan4Life xmen_wolverine end,,2018-02-28 16:04:32,0
9,10,were hiring read about our latest job opening ...,hiring job KellyJobs,,2018-02-28 16:00:46,0


In [389]:
pd.read_sql_query("SELECT * from Movie_tweet",con=conn)

,movie_name,tweet_id
0,Logan,1
1,Logan,2
2,Logan,3
3,Logan,4
4,Logan,5
5,Logan,6
6,Logan,7
7,Logan,8
8,Logan,9
9,Logan,10


In [390]:
pd.read_sql_query("SELECT * from Users",con=conn)

,user_id,screen_name
0,738139568076001280,AarontheWise
1,1398267104,agent_joechavez
2,143545821,canaltn8
3,959474670864158720,Favs_Pops
4,211299432,scedydesigner
5,119591408,tmj_UT_LABOR
6,833612154,APEntertainment
7,117789928,tmj_ut_intern
8,3018132191,steffimaus81
9,149112383,tmj_UT_transp


In [391]:
pd.read_sql_query("SELECT * from user_posts",con=conn)

,user_id,tweet_id
0,738139568076001280,1
1,1398267104,2
2,143545821,3
3,959474670864158720,4
4,211299432,5
5,119591408,6
6,833612154,7
7,117789928,8
8,3018132191,9
9,149112383,10


In [392]:
pd.read_sql_query("SELECT * from Tweets_tags",con=conn)

,Tweet_id,tags
0,1,
1,1,sasori_113
2,1,larajillmiller
3,1,missnotyou
4,1,matthewbouman1
5,1,metalskull423
6,1,brutalpuncher1
7,1,jjravenation52
8,1,davidcart11
9,2,dareto


# Use- cases

# 1. Retrieve the tags associated with a Movie

In [395]:
#For each movie, find the tweets associated with it and retrieve tags from each tweet for a movie
pd.read_sql_query("Select m.movie_name as 'Movie',group_concat(t.tags) as 'Tags' FROM Movie_tweet as m INNER JOIN Tweets_tags as t ON m.tweet_id==t.Tweet_id GROUP BY m.movie_name",con=conn)


,Movie,Tags
0,120battementsparminute,"120battementsparminute,cgr_torcy_mlv,les_cesar..."
1,12Strong,",12strong,,,,12strong,10thoffer,12strong,exams..."
2,47MetersDown,",47metersdown,,47metersdown,mandymoore,,47mete..."
3,ABadMomsChristmas,"abadmomschristmas,kennyg,lebatardshow,,abadmom..."
4,AGhostStory,"aghoststory,criterion,,aghoststory,,aghoststor..."
5,Ak-Nyeo,",,,djregular"
6,Alien,"alien,multistagecorre,ovni,ufo,alien,alienanth..."
7,AllEyezonMe,"alleyezonme,fullsail,,2pac,alleyezonme,blackan..."
8,AlltheMoneyintheWorld,"allthemoneyintheworld,lil_lilium,,allthemoneyi..."
9,AmericanAssassin,",kylemillsauthor,dogmagayle,dylanobriencom,mit..."


# 2. What users are like other users?

In [450]:
#If two users have posted about similar topics they are similar. I have retrieved the tags from each users posts and compared to see if they have posted about similar topics

user1_posts=pd.read_sql("Select u.user_id,t.tags FROM Tweets_tags as t INNER JOIN user_posts as u ON u.tweet_id==t.Tweet_id AND u.user_id=='959474670864158720'",con=conn)
user2_posts=pd.read_sql("Select u.user_id,t.tags FROM Tweets_tags as t INNER JOIN user_posts as u ON u.tweet_id==t.Tweet_id AND u.user_id=='119591408'",con=conn)
user1_tags=user1_posts.tags.unique()
user2_tags=user2_posts.tags.unique()
common_tags=''
flag=0
for item in user1_tags:
    if item in user2_tags:
        flag=1
        common_tags=common_tags+item+','
        
if flag==0:
    print "They do not have anything in common"
else:
    print "They are similar, they have posted about",common_tags

They are similar, they have posted about logan,


# 3. The most popular movies on Twitter 

In [409]:
#Movies who's posts were retweeted the most in the entire database, is what is the most popular
pd.read_sql_query("Select mt.movie_name,SUM(retweet_count) as 'Total retweets' FROM Movie_tweet as mt INNER JOIN Tweets as t ON mt.tweet_id==t.tweet_id GROUP BY mt.movie_name ORDER BY SUM(retweet_count) desc LIMIT 3",con=conn)


,movie_name,Total retweets
0,Thor,244740
1,BlackPanther,226649
2,AmericanAssassin,222927


# 4. The movie that was Trending on Twitter on Feb 27th

In [424]:
#Movies who's posts were retweeted the most on a particular day was trending on that day
pd.read_sqlquery("Select mt.movie_name,SUM(retweet_count) as 'Total retweets' FROM Movie_tweet as mt INNER JOIN Tweets as t ON mt.tweet_id==t.tweet_id and t.created_date LIKE '%2018-02-27%' GROUP BY mt.movie_name ORDER BY SUM(retweet_count) desc LIMIT 1",con=conn)


,movie_name,Total retweets
0,BabyDriver,108209


# All tables are Normalized


    The primary key of the Movies table is movie_name. Every movie name is unique, used to distinguish it. The table has atomic values in every column, and no two columns store the same values. All the non-key attributes are completely dependent on the movie. There are no transitive dependencies.

    The primary key of the Genres table is genre_name. Every genre name is unique, used to distinguish it. The table has atomic values in every column, and no two columns store the same values. 

    The primary key of the Movie_genres table is a composite key of movie_name and genre_name. The pair of movie and a particular genre is unique. The table has atomic values in every column, and no two columns store the same values.
    
    The primary key of the Actors table is the actor_name. Every actor has a unique name, used to distinguish him/her. The table has atomic values in every column, and no two columns store the same values. 
    
    The primary key of the Movie_actors table is a composite key of movie_name and actor_name. The pair of movie and an actor is unique. The table has atomic values in every column, and no two columns store the same values.

    The primary key of the Tweets table is Tweet_id. Every Tweet has a unique ID, used to distinguish it. The table has atomic values in every column, and no two columns store the same values. All the non-key attributes are completely dependent on the Tweet. There are no transitive dependencies.

    The primary key of the Movie_tweet table is a composite key of Tweet_id and movie_name. The pair of movie and a particular Tweet is unique. The table has atomic values in every column, and no two columns store the same values.
    
    The primary key of the Users table is user_id. Every user has a unique ID, used to distinguish him/her. The table has atomic values in every column, and no two columns store the same values. The Twitter screen name is completely dependent on the User ID. 
    
    The primary key of the User_posts table is ID. The pair of user and a particular Tweet is unique . The table has atomic values in every column, and no two columns store the same values.
    
    The primary key of the Tweet_tags table is ID. The pair of tweet and the tags is unique . The table has atomic values in every column, and no two columns store the same values.
    
    